# Explore the neighborhood of song recommendations

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pandas as pd
import numpy as np
import matplotlib as plt
import importlib

import mpd

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline
# change default plot size
plt.rcParams['figure.figsize'] = (15,10)

## Load and prep data

* Load the full data set
* Vectorize the playlists into sparse vectors
* Extract the vocabulary with tid to allow translation back to track_uri

In [ ]:
mpd_all=mpd.load(spark, "onebig", 1)

Extract features from mpd with each playlist being a sparse vector of tracks.  The vector index is the popularity of the song in the global playlist collection with the index value being the appearance count of the track in the playlist.

This count is ignored and treated as 1.0 in track-based playlist searches because the goal is simply to find playlists that share common tracks.  That is, we treat the playlists as the user matrix with each track as a record of a "purchased" item.

The vectorized playlist are limited to the 2^18 most popular songs, the default for CountVectorizer().

In [ ]:
model, result = mpd.vectorizecol(mpd_all.select("pid", "tracks.track_uri"), "track_uri", "features")

In [ ]:
importlib.reload(mpd)

In [ ]:
vdf = mpd.buildvocabdf(spark, model.vocabulary)

In [ ]:
vdf.show(5)

Demonstrate mapping of tracks in voabulary to human readable names from mpd dataset

In [ ]:
from pyspark.sql.functions import explode
tname=mpd_all.select(explode("tracks").alias("tracks")).select("tracks.track_name", "tracks.track_uri", "tracks.artist_name").distinct()

In [ ]:
vdf.join(tname, tname.track_uri == vdf.term).drop(vdf.term).orderBy("tid").show(5)

## Prepare data for kNN search

* Prep dataset with vector length to elliminate empty playlists from minHash input

In [ ]:
from pyspark.ml.feature import MinHashLSH

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

vectorlength = udf(lambda x: x.numNonzeros(), IntegerType())

In [ ]:
r2=result
r2=r2.withColumn("vlen", vectorlength(r2.features))

Consider only playlists with one or more entries because MinHash can't handle empty vectors.

In [ ]:
sparsevec = r2.where(r2.vlen > 0)

Sample only 10% of the mpd to build the characteristic matrix for the initial search space in consideration of performance.

In [ ]:
sparsevec= sparsevec.rdd.sample(False, .01, 1).toDF()

Build the signature matrix with min hashing.  Each signature has 5 hashes.

In [ ]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)

In [ ]:
model = mh.fit(sparsevec)

In [ ]:
transformA = model.transform(sparsevec)

The signature vectors are the hashes column.  Together with the playlist id they make up the ~1,000,000x5 signature matrix.

## Explore results for full playlist match

Grab the first playlist in the mpd as a test list (it is very unlikely to be in the sampled set above).

In [ ]:
testpl=result.select("features").rdd.map(lambda x: x.features).take(1)[0]

Find 100 nearest neighbors to get enough playlists to ensure we have 500 tracks to recommend.  On average, 100 playlists should contain about 660 tracks.

In [ ]:
print("Approximately searching dfA for 100 nearest neighbors of the key:")
k100nn = model.approxNearestNeighbors(transformA, testpl, 100)

Visualize the distribution of distances in the 100 neighbors.  Most are far away but some are pretty close.  Given we are only getting 100 near playlists we are naively dictating that our cluster size is 100 simply to make sure we have enough tracks in teh cluster.

In [ ]:
mpd.plothist(k100nn, "distCol", 11)

In [ ]:
k100nn.printSchema()

### Count the popularity of tracks in knn cluster playlists

This follows the inverse decay shape of global track/artist popularity.

It seems logical to assume that track popularity within similar playlists is a good (if basic) foundation for recommended tracks.

In [ ]:
k100nntracks=k100nn.select(explode("track_uri").alias("track_uri"))

In [ ]:
trackrank = k100nntracks.select("track_uri").groupby("track_uri").count().sort(f.col("count").desc())

In [ ]:
trackrank.printSchema()

In [ ]:
importlib.reload(mpd)

In [ ]:
mpd.scatterplotfreq(trackrank)

## Explore results for a subset of playlist

In [ ]:
testpl

Sparse vectors have an [indices method](http://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.linalg.SparseVector.indices) to get the array elements with values.

In [ ]:
testpl.indices

Getting a subset of the playlist is easy by looking at the indices
They are returned as a numpy array. 
[numpy has a built in to chose a random sample from an array](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html)

In [ ]:
rand5npa = np.random.choice(testpl.indices, 5, replace=False)

Note, the machine learning libararies expect sparse vectors of the new ml.linalg package not the mllib.linalg package.  There is a [conversion method for the old format to return as the new ML format](http://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.linalg.SparseVector.asML).

In [ ]:
from pyspark.ml.linalg import SparseVector 

In [ ]:
np.sort(rand5npa)

In [ ]:
print(testpl.size, np.sort(rand5npa), np.ones(len(rand5npa)))

Use np.sort and np.ones to help create new spase vector for the search.

The sampled playlist still exists in the dimensionality of the of the original testpl.

In [ ]:
rand5pl = SparseVector(testpl.size, np.sort(rand5npa), np.ones(len(rand5npa)))

In [ ]:
rand5pl

### Search for the k=100 neighbors for the 5-element playlist

In [ ]:
k100nn5seed = model.approxNearestNeighbors(transformA, rand5pl, 100)

In [ ]:
k100nn5seed.count()

Note we're only getting 5 candidates so that must mean we [don't have enough candidates in the hash bucket](https://spark.apache.org/docs/2.2.0/ml-features.html#approximate-nearest-neighbor-search).  Wonder if increasing the hash number in the minhash will improve this.

In [ ]:
mpd.plothist(k100nn5seed, "distCol", 11)

## Explore impact of choice on construction of sampled playlist

I would have expected more results not less from a smaller playlist.
I'm assuming that a smaller set of tracks would match a larger collection of playlists.

This could be a luck of the draw for the random selection. Repeating the random choice selection from the playlist now gives me 11 results.

So this is challenging. How to ensure I get enough results from the search

I assume what is happening is that if there are tracks that are less popular in the search then I'm going to be more of an edge case playlist.

Yeah, and if i take just the top-5 most popular songs in the playlist then i get my 100 results as expected.

Randomly choose tracks from playlist. Leads to small result set for near neighbors

In [ ]:
rand5npa = np.random.choice(testpl.indices, 5)

Choose 5 most popular songs from playlist. Leads to high number of near neighbors.

In [ ]:
rand5npa = np.sort(testpl.indices)[0:5]

Choose the first 5 songs from the playlist. This is the continuation model.  You hear 5 songs and now recommend the rest.

Note, in the current construction of testpl we have lost the orignal playlist order since we are deriving this from the sparse vector.

Really need to go back and redefine testpl as coming from the original playlist

In [ ]:
rand5npa = testpl.indices[0:5]

In [ ]:
rand5pl = SparseVector(testpl.size, np.sort(rand5npa), np.ones(len(rand5npa)))

In [ ]:
rand5pl

### Longer signature matrix doesn't create larger clusters

In [ ]:
mh10 = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=10)

In [ ]:
model10 = mh10.fit(sparsevec)

In [ ]:
transformA10 = model10.transform(sparsevec)

This search is with a top 5 tracks selected sample playlist and a larger signature matrix.  The higher hit count of neighbors comes from the more popular track selection for the search.

In [ ]:
k100nn5seed10 = model.approxNearestNeighbors(transformA10, rand5pl, 100)

In [ ]:
k100nn5seed10.count()

Even though we have more neighbors with the same small search playlist, most of those neigbors are distant. This makes sense because we are looking for similarity on a small cross section of each playlist.

This is somewhat akin to the banding method for speeding LSH with minhash signature matrix.  If our search term is a band then we have matched a number of playlists that share these tracks in that band.

In [ ]:
mpd.plothist(k100nn5seed10, "distCol", 11)

## Explore artists as clustering

Artists make an inherently good clustering since the naturally and logically shrink the collection of tracks.  They also provide the potential for developing a utility matrix with explicit ratings since the same artist is much more likely to appear multiple times in the same playlist, suggesting a preference for the artist.

The artist utility matrix is built from the original mpd.

In [ ]:
amodel, aresult = mpd.vectorizecol(mpd_all.select("pid", "tracks.artist_uri"), "artist_uri", "features")

In [ ]:
avdf = mpd.buildvocabdf(spark, amodel.vocabulary)

In [ ]:
avdf.show(5)

Review artist names to get a sense of the most popular.

In [ ]:
aname=mpd_all.select(explode("tracks").alias("tracks")).select("tracks.artist_uri", "tracks.artist_name").distinct()

In [ ]:
avdf.join(aname, aname.artist_uri == avdf.term).drop(avdf.term).orderBy("tid").show(5)

The artist-based playlists are also not likely to have empty vectors since they fall under the 2^18 default.

In [ ]:
ar2=aresult
ar2=ar2.withColumn("vlen", vectorlength(ar2.features))

In [ ]:
ar2.count()

In [ ]:
asparsevec = ar2.where(ar2.vlen > 0)

In [ ]:
asparsevec.count()

Work with a 10% sample of the artist-based playlists.

In [ ]:
asparsevec= asparsevec.rdd.sample(False, .01, 1).toDF()

Build the signature matrix for artists.

In [ ]:
amh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)

In [ ]:
amhmodel = amh.fit(asparsevec)

In [ ]:
amhtransform = amhmodel.transform(asparsevec)

Build a sample search playlist for artists.

In [ ]:
atestpl=aresult.select("features").rdd.map(lambda x: x.features).take(1)[0]

In [ ]:
atestpl

Search for artists playlists similar to the search playlist.

In [ ]:
print("Approximately searching dfA for 100 nearest neighbors of the artist:")
ak100nn = model.approxNearestNeighbors(amhtransform, atestpl, 100)

There is much better similarity across artist based playlists.

This distribution is simlar to when we use the whole playlist to search the playlist space.  This suggests that better coverage of the playlist and tracks will build a better neighbor hood with gradually increasing distance.

In [ ]:
mpd.plothist(ak100nn, "distCol", 11)

Count the artist popularity in the k=100 neighborhood

In [ ]:
k100nnartists=ak100nn.select(explode("artist_uri").alias("artist_uri"))

In [ ]:
artistrank = k100nnartists.select("artist_uri").groupby("artist_uri").count().sort(f.col("count").desc())

In [ ]:
mpd.scatterplotfreq(artistrank)

### Explore subset of playlist match with artists

In [ ]:
atestpl

Get 5 elements from the artist search vector.  Again the most popular due to the construction of the sparse vector.

In [ ]:
arand5npa = np.random.choice(atestpl.indices, 5, replace=False)

In [ ]:
arand5npa

In [ ]:
arand5pl = SparseVector(atestpl.size, np.sort(arand5npa), np.ones(len(arand5npa)))

In [ ]:
ak100nn5seed = model.approxNearestNeighbors(amhtransform, arand5pl, 100)

In [ ]:
ak100nn5seed.count()

In [ ]:
ak100nn5seed.printSchema()

In [ ]:
ak100nn5seed.orderBy("pid").orderBy("distCol").show(5)

So the playlist name matches are less perfect than with the full playlist search, at least based on superficial name matching.

Get the sense that searching for knn with the full sample playlist by tracks got many similarly named playlists.

In [ ]:
ak100nn5seed.join(mpd_all.select("pid", "name"), mpd_all.pid == ak100nn5seed.pid).drop(mpd_all.pid).select("pid", "name").show(5)

In [ ]:
ak100ordered = ak100nn5seed.orderBy("pid").orderBy("distCol")

Looking at the output of the matches sorted by distance (in contract to the random result order above) does give a little better match to the theme of "throwback" even though the words are identical.

In [ ]:
ak100ordered.join(mpd_all.select("pid", "name"), mpd_all.pid == ak100ordered.pid).drop(mpd_all.pid).select("pid", "name", "distCol").orderBy("distCol").show(5)

## Explore knn with full vocab and sample playist

See how it works to search against a larger playlist database.  Do small samples of just 5 songs return larger neighborhood.

In [ ]:
importlib.reload(mpd)

In [ ]:
model, result = mpd.vectorizecol(mpd_all.select("pid", "tracks.track_uri"), "track_uri", "features", size=1188873 )

In [ ]:
mhall = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)

In [ ]:
modelall = mhall.fit(sparsevec)

In [ ]:
transformall = modelall.transform(sparsevec)

In [ ]:
k100nn5seedall = modelall.approxNearestNeighbors(transformall, rand5pl, 100)

In [ ]:
k100nn5seedall.count()

With the full playlist collection of all songs that have more than one occurance the neighborhood results are much better. The search performance doesn't seem to have changed much either. 

In [ ]:
mpd.plothist(k100nn5seedall, "distCol", 11)

In [ ]:
k100nn5seedall.join(mpd_all.select("pid", "name"), mpd_all.pid == k100nn5seedall.pid).drop(mpd_all.pid).select("pid", "name").show(5)

In [ ]:
k100nn5seedall.printSchema()

In [ ]:
k100nn5seedall = k100nn5seedall.withColumnRenamed("pid", "recpid")

In [ ]:
k100nn5seedall.join(mpd_all.select("pid", "name"), mpd_all.pid == k100nn5seedall.recpid).drop(mpd_all.pid).select("recpid", "name", "distCol").orderBy("distCol").show(5)

## Explore quality of music recommendation

Get top 500 recommended songs from 100 recommended playlists based on 5 tracks.

In [ ]:
k100nn5seedall.printSchema()

### Get the ranked resutls of tracks from the recommended neighboring playlists.

In [ ]:
trackrank=k100nn5seedall.select(explode(k100nn5seedall.track_uri).alias("track_uri")).groupBy("track_uri").count().sort(f.col("count").desc())

In [ ]:
trackrank.show(5)

In [ ]:
Y=trackrank.select("count").toPandas()

In [ ]:
X=pd.DataFrame({'X': range(1,Y.size+1,1)})

In [ ]:
plt.pyplot.scatter(X,Y)

### Get global track info from knn recommended neighbors

In [ ]:
k100nn5seedall = k100nn5seedall.withColumnRenamed("pid", "recpid")

In [ ]:
pln=k100nn5seedall.join(mpd_all, mpd_all.pid == k100nn5seedall.recpid)

In [ ]:
pln.printSchema()

In [ ]:
pln.select("name", "modified_at", "num_edits", "num_followers","num_tracks").show()

In [ ]:
pDF=pln.select("pid", explode("tracks").alias("track")).select("track.*")

In [ ]:
pDF.select("track_name").show()

In [ ]:
pDF.count()

### Get knn playlist and global ranking

Get the global track vocabulary, where the track id (tid) is the rank of the song in the global dataset.

In [ ]:
tvdf = mpd.buildvocabdf(spark, model.vocabulary)

In [ ]:
tvdf.printSchema()

In [ ]:
tvdf.count()

Add the global track rank column (track id) from the global track vocabulary.

In [ ]:
grank=trackrank.join(tvdf, trackrank.track_uri == tvdf.term)

In [ ]:
grank.printSchema()

In [ ]:
grank.orderBy("count").show(5)

In [ ]:
grank.count()

In [ ]:
recommendrank=grank.orderBy(f.desc("count"), f.desc("tid")).select("track_uri", "count", "tid")

In [ ]:
recommendrank.count()

Create a row index in recommendation order.  

Can't use monotonically increasing by itself because it skips numbers. Need to [convert to a an increase by one id](https://stackoverflow.com/a/48211877). 

In [ ]:
recommendrank=recommendrank.withColumn("mid", f.monotonically_increasing_id())

In [ ]:
from pyspark.sql.window import Window as W

In [ ]:
windowSpec = W.orderBy("mid")

In [ ]:
recommendrank = recommendrank.withColumn("rank", f.row_number().over(windowSpec)).drop("mid")

In [ ]:
recommendrank.show(10)

In [ ]:
recommendrank.describe("rank").show()

In [ ]:
rand5pl

In [ ]:
testpl

[Convert a numpy array to a dataframe via a pandas dataframe](https://stackoverflow.com/a/46818308)

In [ ]:
rand5pl.indices

In [ ]:
type(rand5pl.indices)

In [ ]:
r5pldf = spark.createDataFrame(pd.DataFrame(rand5pl.indices), ["idx"])

In [ ]:
r5pldf.show()

The search tracks make up half the top 10 popularity of the knn trackrank results.

In [ ]:
recommendrank.join(r5pldf, recommendrank.tid == r5pldf.idx).drop(r5pldf.idx).show()

Want to see the intersection of the recommended tracks and the tracks that were held out of the search playlist.

This is hard to express in sql because a not-equal between columns doesn't reflect the semantics of not in.  That just elliminates the search playlist tracks from the recommended result.

`recommendrank.join(r5pldf, recommendrank.tid != r5pldf.idx)`

It might be quick to eliminate the search tracks from the full playlist with a dot product?  No cause that sums them up.  What i really want is a mask.  Feel this should be easier with sql.

There is a [not in clause with negation ~](https://stackoverflow.com/a/40607572)

In [ ]:
recommendrank.where(~recommendrank.tid.isin(rand5pl.indices.tolist())).show(5)

Subtracting the search playlist from full playlist should get me an index I can intersect with the ranked recommended tracks and see how the real playlist compares the recommended tracks.

This would work but mainly need to get the testpl in a dataframe and then do a not in join.

In [ ]:
plidx = spark.createDataFrame(pd.DataFrame(testpl.indices), ["idx"])

In [ ]:
plidx.show(5)

In [ ]:
recidx = plidx.where(~plidx.idx.isin(rand5pl.indices.tolist()))

In [ ]:
type(recidx)

In [ ]:
recidx.show(5)

In [ ]:
recidx.count()

So of the remaining tracks in the playlist only about 2/3 are covered in the top 500 continuation track recommendations. This means the recommendation model based on popularity isn't great.  Also we missing about 11 tracks that didn't show up recommended tracks at all.

In [ ]:
remainingrank = recommendrank.join(recidx, recidx.idx == recommendrank.tid).drop(recidx.idx).orderBy("rank")

In [ ]:
remainingrank.count()

In [ ]:
remainingrank.show(35)

In [ ]:
mpd.plothist(remainingrank, "rank", 11)